# huawei cup - siamese-Triplet test

使用ipynb笔记本来对python代码进行逐步调试处理/训练

首先，读取数据。

In [1]:
import logging
import os

import numpy as np
import torch

from dissimilarity import calculate_cs_dissimilarity_matrix
import dataset

## 1. 数据读入及预处理
使用 huawei 杯官方给的示例代码，导入数据。

准备数据集，首先定义有关函数。

需要注意，我们的数据集每一个对应着不同的扇区，这里先导一个，每个文件导出的时候也对应不同扇区，训练三个模型。

In [2]:
print("<<< Welcome to 2024 Wireless Algorithm Contest! >>>\n")
## 不同轮次的输入数据可放在不同文件夹中便于管理，这里用户可以自定义
PathSet = {0: "./Test", 1: "./CompetitionData1", 2: "./CompetitionData2", 3: "./CompetitionData3"}
PrefixSet = {0: "Round0", 1: "Round1", 2: "Round2", 3: "Round3"}

Ridx = 1  # 设置比赛轮次索引，指明数据存放目录。0:Test; 1: 1st round; 2: 2nd round ...
PathRaw = PathSet[Ridx]
Prefix = PrefixSet[Ridx]

# 查找文件夹中包含的所有比赛/测试数据文件，非本轮次数据请不要放在目标文件夹中
files = os.listdir(PathRaw)
names = []

for f in sorted(files):
    if f.find('CfgData') != -1 and f.endswith('.txt'):
        names.append(f.split('CfgData')[-1].split('.txt')[0])

## 创建对象并处理
## 这里我们读取 输入1
na = str(1)
FileIdx = int(na)
print('Processing Round ' + str(Ridx) + ' Case ' + str(na))

# 读取配置文件 RoundYCfgDataX.txt
print('Loading configuration data file')
cfg_path = PathRaw + '/' + Prefix + 'CfgData' + na + '.txt'
bs_pos, tol_samp_num, anch_samp_num, port_num, ant_num, sc_num = dataset.read_cfg_file(cfg_path)

# 读取锚点位置文件 RoundYInputPosX.txt
print('Loading input position file')
anch_pos_path = PathRaw + '/' + Prefix + 'InputPos' + na + '.txt'
anch_pos = dataset.read_anch_file(anch_pos_path, anch_samp_num)

# 读取信道文件 RoundYInputDataX.txt，这里已经读取到了.npy中，直接加载即可
print('Loading input CSI data of ' + 'Case ' + str(na))
csi_file = PathRaw + '/' + Prefix + 'InputData' + na + '.npy'
H = np.load(csi_file)  # 后续可以直接load数据文件

print('Loading succeed')


<<< Welcome to 2024 Wireless Algorithm Contest! >>>

Processing Round 1 Case 1
Loading configuration data file
Loading input position file
Loading input CSI data of Case 1
Loading succeed


## 2. 计算距离

有两个距离可供选择，一个是 *ADP距离* 一个是 *G-ADP距离*

### 2.1. CS距离的计算
CS 距离的定义如下：
$$
d_{\mathrm{CS}, i, j}=\sum_{b=1}^B \sum_{n=1} ^{N_{sub}}\left(1-\frac{\left|\sum_{m=1}^M\left(\tilde{\mathbf{H}}_{b, m, n}^{(i)}\right)^* \tilde{\mathbf{H}}_{b, m, n}^{(j)}\right|^2}{\left(\sum_{m=1}^M\left|\tilde{\mathbf{H}}_{b, m, n}^{(i)}\right|^2\right)\left(\sum_{m=1}^M\left|\tilde{\mathbf{H}}_{b, m, n}^{(j)}\right|^2\right)}\right)
$$
其中，$N_{sub}$ 是子载波数量，$B$ 是端口数，$M$ 是 BS 的天线数量


### 2.2 G-CS计算



In [ ]:
%matplotlib inline

# 初始化日志记录
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# 计算所有点之间的CS距离,得到tol_samp_num*tol_samp_num大小的矩阵

d_cs = calculate_cs_dissimilarity_matrix(H)

# 保存到文件中
adp_file = PathRaw + '/' + Prefix + 'DCS' + na + '.npy'
np.save(adp_file, d_cs)  # 首次读取后可存储为npy格式的数据文件

logging.info("保存成功")


计算CS距离:   0%|          | 0/20000 [00:00<?, ?it/s]